In [56]:
import pandas as pd


In [57]:
df = pd.read_csv('../data/raw/NS-time table for Upward.csv')

In [58]:
df.columns 

Index(['Stations ', 'Stations_code ', 'Train times', '20502', '20602', '20702',
       '20802', '20902', '20104', '20204', '20304', '20404', '20504', '20604',
       '20704', '20804', '20904', '20106', '20206', '20306', '20406', '20506',
       '20606', '20706', '20806', '20108', '20208', '20308', '20408', '20508',
       '20608', '20708', '20808', '20110', '20210', '20310', '20410', '20510',
       '20610', '20710', '20810', '20112', '20212', '20312', '20412', '20512',
       '20612', '21002', '20712', '20812', '20114', '20214', '20314', '20414',
       '20514', '20614', '21004', '20714', '20814', '20116', '20216', '20316',
       '20416', '20616', '20716', '20816', '20218', '20318', '20418'],
      dtype='object')

In [59]:
df.head(3)


,Stations,Stations_code,Train times,20502,20602,20702,20802,20902,20104,20204,...,20116,20216,20316,20416,20616,20716,20816,20218,20318,20418
0,Menilik II Square T22702,NS27,Returning,6:56:20,7:11:20,7:26:20,7:41:20,7:50:20,7:56:20,8:11:20,...,19:58:20,20:13:20,20:33:20,20:53:20,21:13:20,21:33:20,21:53:20,22:13:20,22:33:20,22:56:20
1,Menilik II Square T22706,NS27,Departure,6:55:32,7:10:32,7:25:32,7:40:32,7:49:32,7:55:32,8:10:32,...,19:57:32,20:12:32,20:32:32,20:52:32,21:12:32,21:32:32,21:52:32,22:12:32,22:32:32,22:55:32
2,Menilik II Square T22706,NS27,Arrival,6:54:57,7:09:57,7:24:57,7:39:57,7:48:57,7:54:57,8:09:57,...,19:56:57,20:11:57,20:31:57,20:51:57,21:11:57,21:31:57,21:51:57,22:11:57,22:31:57,22:54:57


In [60]:
threshold = 0.9  # 50%
nan_cols = df.columns[df.isna().mean() > threshold]
print(nan_cols)
#drop nan_cols
df = df.drop(columns=nan_cols)


Index([], dtype='object')


In [61]:
# raname columns for better readability
df = df.rename(columns={'Stations_code ': 'station_id'})
df = df.rename(columns={'Train times': 'arrival_departure'})
df = df.rename(columns={'Stations ': 'Stations'})



In [62]:
import pandas as pd

# Load data

# Step 1: Add Arrival/Departure column based on row order
df['arr_dep'] = ['Arrival' if i % 2 == 0 else 'Departure' for i in range(len(df))]

# Step 2: Melt train columns
id_vars = ['Stations', 'station_id', 'arrival_departure']
value_vars = [col for col in df.columns if col not in id_vars]
df_melted = df.melt(id_vars=id_vars, value_vars=value_vars,
                    var_name='train_id', value_name='time')

# Step 3: Pivot so we have Arrival & Departure per station/train
df_pivot = df_melted.pivot_table(
    index=['Stations', 'station_id', 'train_id'],
    columns='arrival_departure',
    values='time',
    aggfunc='first'
).reset_index()

# Optional: parse time column to datetime
df_pivot['Arrival'] = pd.to_datetime(df_pivot['Arrival'], errors='coerce')
df_pivot['Departure '] = pd.to_datetime(df_pivot['Departure '], errors='coerce')
df_pivot= df_pivot.rename(columns={'Departure ': 'Departure'})

# Done
print(df_pivot.head())


arrival_departure Stations station_id train_id             Arrival  \
0                    Abnet       NS22    20104 2025-07-25 07:41:43   
1                    Abnet       NS22    20106 2025-07-25 09:43:43   
2                    Abnet       NS22    20108 2025-07-25 11:43:43   
3                    Abnet       NS22    20110 2025-07-25 13:43:43   
4                    Abnet       NS22    20112 2025-07-25 15:43:43   

arrival_departure           Departure Returning   
0                 2025-07-25 07:42:08        NaN  
1                 2025-07-25 09:44:08        NaN  
2                 2025-07-25 11:44:08        NaN  
3                 2025-07-25 13:44:08        NaN  
4                 2025-07-25 15:44:08        NaN  


C:\Users\nebiy\AppData\Local\Temp\ipykernel_16116\3484329245.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_pivot['Arrival'] = pd.to_datetime(df_pivot['Arrival'], errors='coerce')
C:\Users\nebiy\AppData\Local\Temp\ipykernel_16116\3484329245.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_pivot['Departure '] = pd.to_datetime(df_pivot['Departure '], errors='coerce')


In [63]:
df_pivot['Dwell_Time'] = df_pivot['Departure'] - df_pivot['Arrival']

In [64]:
df_pivot.sort_values(by=['Dwell_Time', 'Arrival'], inplace=True)


In [65]:
df_pivot.head()

arrival_departure,Stations,station_id,train_id,Arrival,Departure,Returning,Dwell_Time
98,Abo,NS7,20502,2025-07-25 06:03:04,2025-07-25 06:03:29,NaN,0 days 00:00:25
1170,Saris,NS8,20502,2025-07-25 06:06:22,2025-07-25 06:06:47,NaN,0 days 00:00:25
105,Abo,NS7,20602,2025-07-25 06:18:04,2025-07-25 06:18:29,NaN,0 days 00:00:25
567,Lancha,NS12,20502,2025-07-25 06:18:13,2025-07-25 06:18:38,NaN,0 days 00:00:25
1505,Temenja Yazh,NS13,20502,2025-07-25 06:20:03,2025-07-25 06:20:28,NaN,0 days 00:00:25


In [66]:
df_pivot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1541 entries, 98 to 1540
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   Stations    1541 non-null   object         
 1   station_id  1541 non-null   object         
 2   train_id    1541 non-null   object         
 3   Arrival     1452 non-null   datetime64[ns] 
 4   Departure   1452 non-null   datetime64[ns] 
 5   Returning   67 non-null     object         
 6   Dwell_Time  1452 non-null   timedelta64[ns]
dtypes: datetime64[ns](2), object(4), timedelta64[ns](1)
memory usage: 96.3+ KB


In [67]:
counts = df_pivot.groupby('Stations').size()
print(counts[counts <= 1])  # stations with 1 or 0 records


Series([], dtype: int64)


In [68]:
#save df_privot as csv
df_pivot.to_csv('../data/processed/NS_time_table_processed.csv', index=False)

In [69]:
import pandas as pd
from datetime import datetime

# Assuming your data is loaded as `df` with columns: Stations, Arrival, Departure, etc.
# Example structure:
# df = pd.read_csv("your_data.csv")


# Sort by Stations and Arrival Time
df_sorted = df_pivot.sort_values(['Stations', 'Arrival'])

# Calculate Inter-Arrival Gaps (time between consecutive trains at the same station)
df_sorted['Next_Arrival'] = df_sorted.groupby('Stations')['Arrival'].shift(-1)
df_sorted['Arrival_Gap'] = (df_sorted['Next_Arrival'] - df_sorted['Arrival']).dt.total_seconds() / 60  # in minutes

# Drop NaN (last train at each station has no "next arrival")
df_gaps = df_sorted.dropna(subset=['Arrival_Gap'])

# Aggregate by Stations
station_stats = df_gaps.groupby('Stations')['Arrival_Gap'].agg(
    Avg_Gap='mean',
    Min_Gap='min',  # Busiest station (shortest gap)
    Max_Gap='max',  # Least busy station (longest gap)
    Gap_Std='std'   # Consistency of gaps
).reset_index()

# Sort by Avg_Gap to see busiest stations first
station_stats = station_stats.sort_values('Avg_Gap')

print(station_stats)

                    Stations    Avg_Gap   Min_Gap  Max_Gap   Gap_Std
7              Kality T20618  14.764103  5.866667     23.0  3.314762
0                      Abnet  14.769231  6.000000     23.0  3.306099
2                 Adey Ababa  14.769231  6.000000     23.0  3.306099
1                        Abo  14.769231  6.000000     23.0  3.306099
3             Atikilt Tera    14.769231  6.000000     23.0  3.306099
4               Autobus Tera  14.769231  6.000000     23.0  3.306099
5              Darmar T22010  14.769231  6.000000     23.0  3.306099
6              Gojam Berenda  14.769231  6.000000     23.0  3.306099
8                     Lancha  14.769231  6.000000     23.0  3.306099
9                     Leghar  14.769231  6.000000     23.0  3.306099
10  Menilik II Square T22706  14.769231  6.000000     23.0  3.306099
11               Meshoulekya  14.769231  6.000000     23.0  3.306099
12                    Mexico  14.769231  6.000000     23.0  3.306099
13              Nefas Silk 1  14.7

In [72]:
import matplotlib
matplotlib.use('Agg')  # Use the 'Agg' backend which is always available
# Now import pyplot
import matplotlib.pyplot as plt

ValueError: Key backend: 'module://matplotlib_inline.backend_inline' is not a valid value for backend; supported values are ['gtk3agg', 'gtk3cairo', 'gtk4agg', 'gtk4cairo', 'macosx', 'nbagg', 'notebook', 'qtagg', 'qtcairo', 'qt5agg', 'qt5cairo', 'tkagg', 'tkcairo', 'webagg', 'wx', 'wxagg', 'wxcairo', 'agg', 'cairo', 'pdf', 'pgf', 'ps', 'svg', 'template']

In [70]:
#visualize the busiest stations
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plt.barh(station_stats['Stations'], station_stats['Avg_Gap'], color='skyblue')
plt.xlabel('Average Arrival Gap (minutes)')
plt.title('Average Arrival Gap by Station')
plt.show()

ValueError: Key backend: 'module://matplotlib_inline.backend_inline' is not a valid value for backend; supported values are ['gtk3agg', 'gtk3cairo', 'gtk4agg', 'gtk4cairo', 'macosx', 'nbagg', 'notebook', 'qtagg', 'qtcairo', 'qt5agg', 'qt5cairo', 'tkagg', 'tkcairo', 'webagg', 'wx', 'wxagg', 'wxcairo', 'agg', 'cairo', 'pdf', 'pgf', 'ps', 'svg', 'template']